# Использование библиотек MODIN и NUMBA для ускорения работы с данными. Пример работы с kafka.

## MODIN

In [14]:
!sudo pip install modin[dask]

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [15]:
%%time
import pandas 
pandas_csv_data = pandas.read_csv('Building_Permits.csv')

CPU times: user 4.51 s, sys: 1.73 s, total: 6.24 s
Wall time: 5.55 s


In [16]:
%%time
import modin.pandas as pd
modin_csv_data = pd.read_csv("Building_Permits.csv")

CPU times: user 4.51 s, sys: 468 ms, total: 4.98 s
Wall time: 5.37 s


Как видно из строк кода выше, применение MODIN незначительно ускорило загрузку данных из файла.

In [25]:
pandas_csv_data.describe(include='all')

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
count,198900,198900.000000,198900,198900,198900,198900,198900.000000,2216,198900,196132,...,155534.000000,155534,155738.000000,155738,5359,197183.000000,197175,197184.000000,197200,1.989000e+05
unique,181495,NaN,8,1291,4896,1055,NaN,18,1704,21,...,NaN,5,NaN,5,1,NaN,41,NaN,57604,NaN
top,201602179765,NaN,otc alterations permit,09/15/2017,3708,001,NaN,A,Market,St,...,NaN,wood frame (5),NaN,wood frame (5),Y,NaN,Financial District/South Beach,NaN,"(37.79226164705184, -122.4034859571375)",NaN
freq,101,NaN,178844,413,1195,10114,NaN,1501,5443,138358,...,NaN,113350,NaN,114382,5359,NaN,21816,NaN,554,NaN
mean,NaN,7.522323,NaN,NaN,NaN,NaN,1121.728944,NaN,NaN,NaN,...,4.072878,NaN,4.089529,NaN,NaN,5.538403,NaN,94115.500558,NaN,1.162048e+12
std,NaN,1.457451,NaN,NaN,NaN,NaN,1135.768948,NaN,NaN,NaN,...,1.585756,NaN,1.578766,NaN,NaN,2.887041,NaN,9.270131,NaN,4.918215e+11
min,NaN,1.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,...,1.000000,NaN,1.000000,NaN,NaN,1.000000,NaN,94102.000000,NaN,1.293532e+10
25%,NaN,8.000000,NaN,NaN,NaN,NaN,235.000000,NaN,NaN,NaN,...,3.000000,NaN,3.000000,NaN,NaN,3.000000,NaN,94109.000000,NaN,1.308567e+12
50%,NaN,8.000000,NaN,NaN,NaN,NaN,710.000000,NaN,NaN,NaN,...,5.000000,NaN,5.000000,NaN,NaN,6.000000,NaN,94114.000000,NaN,1.371840e+12
75%,NaN,8.000000,NaN,NaN,NaN,NaN,1700.000000,NaN,NaN,NaN,...,5.000000,NaN,5.000000,NaN,NaN,8.000000,NaN,94122.000000,NaN,1.435000e+12


In [18]:
pandas_csv_data.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Street Number',
       'Street Number Suffix', 'Street Name', 'Street Suffix', 'Unit',
       'Unit Suffix', 'Description', 'Current Status', 'Current Status Date',
       'Filed Date', 'Issued Date', 'Completed Date',
       'First Construction Document Date', 'Structural Notification',
       'Number of Existing Stories', 'Number of Proposed Stories',
       'Voluntary Soft-Story Retrofit', 'Fire Only Permit',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units',
       'Plansets', 'TIDF Compliance', 'Existing Construction Type',
       'Existing Construction Type Description', 'Proposed Construction Type',
       'Proposed Construction Type Description', 'Site Permit',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries', 'Zipcode',
       'Location', 'Record ID'],
     

Выполним некоторые операции над данными для сравнения работы MODIN и Pandas.

In [19]:
%%time
import pandas
results_pandas = pandas_csv_data.groupby(by=pandas_csv_data['Zipcode']).sum()

CPU times: user 166 ms, sys: 1.12 s, total: 1.28 s
Wall time: 1.29 s


In [20]:
%%time
results = modin_csv_data.groupby(by=modin_csv_data['Zipcode']).sum()

CPU times: user 2.21 s, sys: 265 ms, total: 2.48 s
Wall time: 2.88 s


In [21]:
%%time 
pandas_csv_data.sort_values(by=['Record ID'])

CPU times: user 190 ms, sys: 59.6 ms, total: 250 ms
Wall time: 238 ms


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
1857,201301168144,8,otc alterations permit,01/16/2013,1731,025,1290,NaN,20th,Av,...,3.0,constr type 3,3.0,constr type 3,NaN,4.0,Sunset/Parkside,94122.0,"(37.763774708907924, -122.47797904331227)",12935321500
3944,201302119878,8,otc alterations permit,02/11/2013,0692,004,1005,NaN,Larkin,St,...,3.0,constr type 3,3.0,constr type 3,NaN,3.0,Nob Hill,94109.0,"(37.78722929603072, -122.41849222292733)",12957222234
4431,201302140271,8,otc alterations permit,02/14/2013,7308,001,310,NaN,Arballo,Dr,...,1.0,constr type 1,1.0,constr type 1,NaN,7.0,Lakeshore,94132.0,"(37.72050631470149, -122.48387775809178)",12962261960
4543,201302150354,4,sign - erect,02/15/2013,0030,001,425,NaN,North Point,St,...,1.0,constr type 1,NaN,NaN,NaN,3.0,North Beach,94133.0,"(37.80616170482821, -122.41406722020226)",12963421006
4820,201302200559,8,otc alterations permit,02/20/2013,0052,022,2262,NaN,Mason,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.804554861100236, -122.41310733655916)",12966291016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198880,M893387,8,otc alterations permit,02/23/2018,1841,025D,1442,NaN,Funston,Av,...,NaN,NaN,NaN,NaN,NaN,7.0,Inner Sunset,94122.0,"(37.76117190796411, -122.47007880106999)",1498337116842
198881,M893407,8,otc alterations permit,02/23/2018,0018,004,2300,NaN,Stockton,St,...,NaN,NaN,NaN,NaN,NaN,3.0,North Beach,94133.0,"(37.80747140583783, -122.41024025753224)",1498338347099
198882,201802232172,8,otc alterations permit,02/23/2018,6021,001B,730,NaN,Excelsior,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,11.0,Excelsior,94112.0,"(37.723206757170836, -122.42763728105176)",1498339194341
198884,M893447,8,otc alterations permit,02/23/2018,3504,030,1699,NaN,Market,St,...,NaN,NaN,NaN,NaN,NaN,6.0,Mission,94103.0,"(37.77217130976155, -122.42193576313322)",1498341156230


In [22]:
%%time 
modin_csv_data.sort_values(by=['Record ID'])

CPU times: user 11.6 s, sys: 2.07 s, total: 13.6 s
Wall time: 44.9 s


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
1857,201301168144,8,otc alterations permit,01/16/2013,1731,025,1290,NaN,20th,Av,...,3.0,constr type 3,3.0,constr type 3,NaN,4.0,Sunset/Parkside,94122.0,"(37.763774708907924, -122.47797904331227)",12935321500
3944,201302119878,8,otc alterations permit,02/11/2013,0692,004,1005,NaN,Larkin,St,...,3.0,constr type 3,3.0,constr type 3,NaN,3.0,Nob Hill,94109.0,"(37.78722929603072, -122.41849222292733)",12957222234
4431,201302140271,8,otc alterations permit,02/14/2013,7308,001,310,NaN,Arballo,Dr,...,1.0,constr type 1,1.0,constr type 1,NaN,7.0,Lakeshore,94132.0,"(37.72050631470149, -122.48387775809178)",12962261960
4543,201302150354,4,sign - erect,02/15/2013,0030,001,425,NaN,North Point,St,...,1.0,constr type 1,NaN,NaN,NaN,3.0,North Beach,94133.0,"(37.80616170482821, -122.41406722020226)",12963421006
4820,201302200559,8,otc alterations permit,02/20/2013,0052,022,2262,NaN,Mason,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.804554861100236, -122.41310733655916)",12966291016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198880,M893387,8,otc alterations permit,02/23/2018,1841,025D,1442,NaN,Funston,Av,...,NaN,NaN,NaN,NaN,NaN,7.0,Inner Sunset,94122.0,"(37.76117190796411, -122.47007880106999)",1498337116842
198881,M893407,8,otc alterations permit,02/23/2018,0018,004,2300,NaN,Stockton,St,...,NaN,NaN,NaN,NaN,NaN,3.0,North Beach,94133.0,"(37.80747140583783, -122.41024025753224)",1498338347099
198882,201802232172,8,otc alterations permit,02/23/2018,6021,001B,730,NaN,Excelsior,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,11.0,Excelsior,94112.0,"(37.723206757170836, -122.42763728105176)",1498339194341
198884,M893447,8,otc alterations permit,02/23/2018,3504,030,1699,NaN,Market,St,...,NaN,NaN,NaN,NaN,NaN,6.0,Mission,94103.0,"(37.77217130976155, -122.42193576313322)",1498341156230


In [31]:
%%time 
pandas_csv_data['Street Number Suffix'].fillna('A')

CPU times: user 29.5 ms, sys: 0 ns, total: 29.5 ms
Wall time: 28.3 ms


0         A
1         A
2         A
3         A
4         A
         ..
198895    A
198896    A
198897    A
198898    A
198899    A
Name: Street Number Suffix, Length: 198900, dtype: object

In [32]:
%%time 
modin_csv_data['Street Number Suffix'].fillna('A')

CPU times: user 147 ms, sys: 68 µs, total: 147 ms
Wall time: 144 ms


0         A
1         A
2         A
3         A
4         A
         ..
198895    A
198896    A
198897    A
198898    A
198899    A
Name: Street Number Suffix, Length: 198900, dtype: object

Таким образом, MODIN дает ускорение в работе при загрузке данных, а в операциях, требующих прохода по всем строкам датафрейма, она уступает по времени Pandas.

## NUMBA

In [1]:
!sudo pip install numba

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


**Пример**. Пусть есть матрицы $X,Y \in \mathbb{R}^{N\times D}$. Требуется построить матрицу $Z=z_ij$ $\in \mathbb{R}^{N\times N}$ такую, что $z_{i,j}$ - расстояние между $X[i]$ и $Y[j]$. Сначала проведем рассчет с помощью *Numpy*.

In [2]:
import time
import numpy as np

def pdist_naive (X, Y):
    res = np.empty((X.shape[0], Y.shape[0]))
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            dist = 0.0
            for k in range(X.shape[1]):
                diff = X[i, k] - Y[i, k]
                dist += diff * diff
            res[i, j] = np.sqrt(dist)
    return res


X = np.ones((1000, 2000), np.int64) * 5
Y = np.ones((1000, 2000), np.int64) * 10

start = time.time()
Z = pdist_naive(X,Y)
end = time.time()
run_time = end - start

print("Average time = {}".format(run_time))

Average time = 3582.644556283951


Теперь применим *Numba* и посмотрим, насколько улучшилось время вычислений.

In [4]:
from numba import jit
import time
import numpy as np

@jit (nopython=True)
def pdist_naive (X, Y):
    res = np.empty((X.shape[0], Y.shape[0]))
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            dist = 0.0
            for k in range(X.shape[1]):
                diff = X[i, k] - Y[i, k]
                dist += diff * diff
            res[i, j] = np.sqrt(dist)
    return res


X_n = np.ones((1000, 2000), np.int64) * 5
Y_n = np.ones((1000, 2000), np.int64) * 10

start = time.time()
Z_n = pdist_naive(X,Y)
end = time.time()
run_time_n = end - start

print("Average time = {}".format(run_time_n))

Average time = 4.587498426437378


Время вычислений с использованием *Numba* улучшилось по сравнению с *Numpy* в **800 раз**!

Проверим, совпадают ли результаты.

In [12]:
def check_results(X, Y):
    check = False
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            if Z[i, j] == Z_n[i, j]:
                check = True
            else:
                check = False
                print("Results aren't equal!")
                return
    if check == True:
        print("Results are equal!")
    
    
check_results(Z, Z_n)

Results are equal!


Таким образом, мы видим явное преимущество работы библиотеки *Numba* по сравнению с *Numpy* для реализации вычислений с большим объемом данных.

## Пример работы с Kafka и Zookeeper

Для реализации примера работы с kafka я установила ее на linux, а также поставила zookeeper. Сначала я запустила zookeper, используя команду *./bin/zookeeper-server-start.sh config/zookeeper.properties*

![title](1.jpg)

Затем запустила kafka с помощью *./bin/kafka-server-start.sh config/server.properties*

![title](2.jpg)

Далее я создала продьюсера с помощью *./bin/kafka-concole-producer.sh --broker-list localhost:9092 --topic FirstTopic* и написала несколько строк сообщений.

![title](3.jpg)

Добавила одного консьюмера с помощью *./bin/kafka-concole-producer.sh --bootstrap-server localhost:9092 --topic FirstTopic* и увидела, что сообщения от продьюсера присылаются.

![title](4.jpg)

В конечном итоге, я добавила еще одного консьюмера и убедилась в том, что сообщения от продьюсера доходят и до него.

![title](5.jpg)